<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---


<p align="center"><h1 align="center">Quick Start: Deploy ML Model into REST API that Uses Customized Serverless Function Code </h1> 

---

<h3 align="center">(Deploy a production-ready REST API and serverless function in 10 minutes...)</h3></p>



---

In [ ]:
! pip install aimodelshare --upgrade

# **Deploying an ML Model Using the aimodelshare Library**

> This tutorial will take you through how to deploy a machine learning model on AWS's serverless computing infrastructure using the Python library. 
>
>**This tutorial has five steps:** 
> 1. [Train a Model](#step-1)
> 2. [Save Objects to Local Folder](#step-2)
> 3. [Define Expected REST API Input And Output JSON](#step-3)
> 4. [Write Code to Accept REST API Input and Return Output (Custom Code used in live Serverless Lambda)](#step-4)
> 5. [Deploy Model into REST API On AWS Using AI Model Share Initiative's aimodelshare Library](#step-5)


<a name="step-1"></a>
# **[1] Train Model**

> We will cheat to speed things up rather than training a model.  Let's import a pretrained image classification model.  The model classifies images into five flower categories.


In [2]:
import aimodelshare as ai
keras_model, y_train_labels = ai.import_quickstart_data("flowers")




Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.


<a name="step-2"></a>
# **[2] Prepare Deployment Folder**

> Here we simply create a folder named "deploy" to store any files you need available in your deployment environment (i.e.- your model file and preprocessor). This folder will be uploaded to AWS's serverless computing infrastructure. 
>
>**The steps to prepare the deployment folder consist of:**
> 1. Create Deployment Folder
> 2. Save Model to Deployment Folder

### **[2.1] Create Deployment Folder**

> Create a new folder with a suitable name to store all the required files and folders in.

In [3]:
deployment_folder = 'deploy'

import os
if os.path.isdir(deployment_folder):
    shutil.rmtree(deployment_folder)
os.mkdir(deployment_folder)

### **[2.2] Save Model To Deployment Folder**

> We are using the Onnx Python library to save a model because it only requires a light library to save space in the final serverless function, but you can use Tensorflow or Pytorch among other options.

In [4]:
# convert TensorFlow Keras model to ONNX model
from aimodelshare.aimsonnx import model_to_onnx
onnx_model = model_to_onnx(keras_model,
                           framework='keras',
                           transfer_learning=False,
                           deep_learning=True)

# save the model to the deployment folder
with open(os.path.join(deployment_folder, "runtime_model.onnx"), "wb") as f:
    f.write(onnx_model.SerializeToString())

<a name="step-3"></a>
# **[3] Define Input And Output JSON Formats**

Define the input and output JSON that your REST API requires to successfully process a request through the code in your Serverless function (see example code in final step below!)

The input JSON structure we are using here has 'data' as the key and its value is in the form of a base64 encoded image string.  Base64 encoding allows REST APIs to send files like images as strings.:

In [5]:
input_json_exampledata = {
         "data": "base64 encoded string"
    }

The output JSON structure we are using here is a list with a single string label:

In [6]:
output_json_exampledata = ["label"]

<a name="step-4"></a>
# **[4] Write Serverless Fxn Code to Process Image Input into Output**

> Serverless code will:
> 1. Load the model and code to preprocess/transform image data into the global runtime environment
> 2. Preprocess the input passed
> 3. Pass preprocessed input to model and generate prediction
> 4. Return prediction to output result





In [7]:
custom_lambda_handler_code = """

# Importing all the required libraries
import numpy as np
import pandas as pd
import os
import six
import imghdr
import base64
import json
import onnxruntime as rt
import cv2
import numpy as np

# load the ML model in global environment for faster inference time for subsequent function calls
model_file_path = 'runtime_model.onnx'
model = rt.InferenceSession('./' + model_file_path)

# this is the preprocessor function 
def preprocessor(data, shape=(192, 192)):

    # Resize a color image and min/max transform the image
    img = cv2.imread(data) # read in image from filepath
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of BGR, we convert the order to RGB
    img = cv2.resize(img, shape) # change height and width of image
    img = img / 255.0 # min/max transform

    # Convert cv2 image to NumPy array
    X = np.array(img) # converting to NumPy array
    X = np.expand_dims(X, axis=0) # increase dimensions to convert object shape to [1, h, w, channels]
    X = np.array(X, dtype=np.float32) # reduce precision for faster training and inferencing

    return X

# this is the predict function to preprocess image data and generate predictions from our model
def predict(image_path):

    # labels corresponding to the index that is predicted
    labels = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

    # preprocessing the input data
    input_data = preprocessor(image_path)
    
    # generate prediction
    input_name = model.get_inputs()[0].name
    res = model.run(None, {input_name: input_data})
    prob = res[0]

    # map label to index with highest probability
    index = prob.argmax(axis=-1)
    result = list(map(lambda x: labels[x], index))

    return result

### event['body'] is the input JSON data from REST API request ###

def handler(event, context):

    # Contents of 'body' key is string, converting content to JSON object
    body = event['body']
    if isinstance(body, six.string_types):
        body = json.loads(body)

    # Fetching image in the form of a base64 encoded string stored within 'data' key of body
    data = body['data']

    # Decoding base64 encoded string into required data type
    sample = base64.b64decode(data)

    # Predict function requires path to image file as argument, saving data to file
    temp_file = "/tmp/temp_file"
    with open(temp_file, "wb") as fh:
        fh.write(sample)

    # Calling the  predict function
    result = predict(temp_file)

    # Deleting the image file created by the user for prediction
    os.remove(temp_file)

    # Returning predictions to REST API output
    result_dict = {
        "statusCode": 200,
        "headers": {
            "Access-Control-Allow-Origin" : "*",
            "Access-Control-Allow-Credentials": True,
            "Allow" : "GET, OPTIONS, POST",
            "Access-Control-Allow-Methods" : "GET, OPTIONS, POST",
            "Access-Control-Allow-Headers" : "*"
        },
        "body":  json.dumps(result)
    }
    return result_dict 

"""

In [8]:
#Save lambda serverless function code somewhere besideds "deploy" folder

with open("custom_lambda_handler.py", "w") as handler_file:
    handler_file.writelines(custom_lambda_handler_code)

<a name="step-5"></a>
# **[5] Deploy Model On AWS Using aimodelshare Library**

> We have finally finished modifying the deployment folder. Now we can use the aimodelshare library to deploy our model on AWS's serverless computing infrastructure and generate an API endpoint. Simply set your AWS credentials and then run a single Python function to create your live REST API:
> 1. Set AWS Credentials
> 2. Deploy Model into Scalable REST API

### **[5.1] Setting AWS Credentials**

> The path to the user's AWS credentials, credentials.txt, should be passed as the parameter to set_credentials function.

#### **Need to create a credentials file?**  See [credentials instructions here](https://aimodelshare.readthedocs.io/en/latest/create_credentials.html#create-credentials)




In [9]:
from aimodelshare.aws import set_credentials
set_credentials(credential_file="credentials.txt", type="deploy_model", manual=False)

AI Model Share login credentials set successfully.
AWS credentials set successfully.


### **[5.2] Deploying Model**

> Now that your credentials are set, use the deploy_custom_lambda() function to create your REST API. 

deploy_custom_lambda requires the following arguments:
> 1. input_json_exampldata: the format of the JSON input
> 2. output_json_exampldata: the format of the JSON output
> 3. custom_lambda_filepath: the relative path of the custom lambda handler (serverless function code) file
> 4. deployment_dir: the deployment folder's relative path
> 5. custom_libraries: all the libraries required for running the model

In [ ]:
custom_lambda_filepath = "custom_lambda_handler.py"
deployment_dir = "deploy"
private = "FALSE"
custom_libraries = "onnxruntime,opencv-python,numpy,pandas,pillow"

from aimodelshare.deploy_custom_lambda import deploy_custom_lambda
deploy_custom_lambda(input_json_exampledata,
                     output_json_exampledata,
                     custom_lambda_filepath,
                     deployment_dir,
                     private,
                     custom_libraries=custom_libraries)

**Let's test your new REST API by sending it a base64 encoded image:**
> Note: If you get an API Timeout error, then you may need to wait a few mins for your REST API to be available on AWS.  Once you see information returned, it's live and ready to use.



In [15]:
import base64
import requests
import json

api_url = "https://vnq5u7ryr6.execute-api.us-east-2.amazonaws.com/prod/m"


# Import example flower image and base64 encode
with open("/content/quickstart_materials/example_data/100080576_f52e8ee070_n.jpg", "rb") as image_file:
  encoded_string = base64.b64encode(image_file.read())
  data = json.dumps({"data": encoded_string.decode('utf-8') })

# Set up authorization headers and add token (Your API token is available in your playground page...
# under predict > programmatic)
headers = {"Content-Type": "application/json", "authorizationToken": "eyJraWQiOiIxRHBcL2FMakJvNmozdHRHZFd6dEVEbUR5V0FPN3JtVEVyaHRDRnltQmlVST0iLCJhbGciOiJSUzI1NiJ9.eyJjdXN0b206b3JnYW5pemF0aW9uIjoiQ29sdW1iaWEgVW5pdmVyc2l0eSBDb21wdXRlciBTY2llbmNlIiwic3ViIjoiMTA4YmM2OGQtN2YxYS00MGVmLWJmOWYtMWY1NDFkY2MwNTIwIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC51cy1lYXN0LTIuYW1hem9uYXdzLmNvbVwvdXMtZWFzdC0yX1gzQlRHNzhnMiIsInBob25lX251bWJlcl92ZXJpZmllZCI6ZmFsc2UsImNvZ25pdG86dXNlcm5hbWUiOiJNTF9HZW5vbWljcyIsImF1ZCI6IjI1dnNzYm5lZDJiYmFvaTFxN3JzNGk5MTR1IiwiZXZlbnRfaWQiOiI1MTBiNTc3MC1mYmUwLTQ1YzItODg0OC1hNTgxOGNkYjAwNjciLCJ0b2tlbl91c2UiOiJpZCIsImF1dGhfdGltZSI6MTY2NTU4MjkyOCwibmFtZSI6Ik1hY2hpbmUgTGVhcm5pbmcgZm9yIEZ1bmN0aW9uYWwgR2Vub21pY3MiLCJwaG9uZV9udW1iZXIiOiIrMTIwMzg5MjI4OTAiLCJleHAiOjE2NjU2OTI0NTQsImlhdCI6MTY2NTYwNjA1NCwiZW1haWwiOiJhaW1vZGVsc2hhcmUyQGdtYWlsLmNvbSJ9.fSrxtriNK6FN60LCucta_Z_klGjvIzwDrMT1vTQ4YV7WNRdfG2L5VgTKxbX2lr6FgkNGRMLYjcwajiMnBHO8EB9LCpmz_Jrn--etHJtE5aGqVknIOh0CyhaY3GqTol2SC2vtILgy7ZveFW82Tjm25tbD5TBUWd4j-oNV7Z0oHOLxHS-3ZpQ_W7dCAIhosmhtmeq426wyfcOnpfvFldI9r-B1PrzfHIGXpmkhapJRA9THgK2fOmNcW1uKAKafmlnSRuNcLuHdhh9SaqT9T1KI72Y96uoU5G3F3YAT5NN0VHaNLg_nsThbEWMZrvS56EAGKJXrxhsEhTTnsmkCyDbk1A" }

prediction = requests.request("POST", api_url, headers = headers, data=data)

# Print prediction
json.loads(prediction.text)

{'statusCode': 200,
 'headers': {'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Credentials': True,
  'Allow': 'GET, OPTIONS, POST',
  'Access-Control-Allow-Methods': 'GET, OPTIONS, POST',
  'Access-Control-Allow-Headers': '*'},
 'body': '["daisy"]'}